In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install datasets

     |████████████████████████████████| 4.2 MB 13.9 MB/s 
     |████████████████████████████████| 84 kB 3.4 MB/s 
     |████████████████████████████████| 596 kB 76.1 MB/s 
     |████████████████████████████████| 6.6 MB 54.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 15.3 MB/s 
     |████████████████████████████████| 342 kB 14.0 MB/s 
     |████████████████████████████████| 136 kB 73.1 MB/s 
     |████████████████████████████████| 212 kB 66.1 MB/s 
     |████████████████████████████████| 1.1 MB 69.8 MB/s 
     |████████████████████████████████| 127 kB 90.0 MB/s 
     |████████████████████████████████| 271 kB 89.6 MB/s 
     |████████████████████████████████| 144 kB 97.4 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [ ]:
import pandas as pd
from transformers import T5TokenizerFast, T5ForConditionalGeneration, T5Config, PegasusTokenizerFast, PegasusForConditionalGeneration, BartForConditionalGeneration, BartTokenizerFast,  Trainer, TrainingArguments, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import requests
from urllib.parse import urlparse
import random
from sklearn.model_selection import KFold
from datasets import load_dataset, Dataset
import numpy as np
from datetime import datetime

In [ ]:
torch.manual_seed(0)
random.seed(0)
device = torch.device('cuda')

In [ ]:
filenames = ['summ_data_eng_preprocessed_hashtag_mention_run_removed_mention_replaced.csv']

In [ ]:
def retrieval_eval(query, gold_urls):
  retrieval_1 = 0
  retrieval_5 = 0
  retrieval_10 = 0
  retrieval_20 = 0
  retrieval_50 = 0
  mrr = 0
  reciprocal_ranks = []

  for i in range(0,len(query)):
    added = False
    cand = query[i]
    gold_url = urlparse(gold_urls[i])
    gold_url_netloc = gold_url[1]
    gold_url_path = gold_url[2]
    if gold_url_path[-1]=='/':
      gold_url_path = gold_url_path[:-1]
    
    params = {
            'query': cand.strip(),
            'pageSize': '50',
            'key': '<your_key>',
        }
    try:
        response_dict = requests.get(
            'https://factchecktools.googleapis.com/v1alpha1/claims:search',
            params=params).json()
    except Exception as e:
      response_dict = {}
      print("Google API is not working")
      break

    if 'claims' in response_dict.keys():
      for i in range(0, len(response_dict["claims"])):
        claim = response_dict['claims'][i]
        ret_url = urlparse(claim['claimReview'][0]['url'])
        ret_url_netloc = ret_url[1]
        ret_url_path = ret_url[2]
        if ret_url_path[-1]=='/':
          ret_url_path = ret_url_path[:-1]

        if(ret_url_netloc == gold_url_netloc and ret_url_path == gold_url_path):
          added = True
          reciprocal_ranks.append(1/(i+1))
          if i<1:
            retrieval_1 +=1

          if i<5:
            retrieval_5 +=1

          if i<10:
            retrieval_10 +=1  

          if i<20:
            retrieval_20 +=1

          retrieval_50 +=1
          break
    
    if added == False:
      reciprocal_ranks.append(0)

  if len(reciprocal_ranks) >0:
    mrr = sum(reciprocal_ranks)/len(reciprocal_ranks)

  return { 
      'retrieval_1': (retrieval_1*100)/len(query),
      'retrieval_5': (retrieval_5*100)/len(query),
      'retrieval_10': (retrieval_10*100)/len(query),
      'retrieval_20': (retrieval_20*100)/len(query),
      'retrieval_50': (retrieval_50*100)/len(query),
      'mrr': mrr,
      }

In [ ]:
def pegasus_summarizer(df,model, min_len, max_len, type):
  pegasus_summaries = []
  for row in df:
    preprocess_text = row.strip().replace("\n","")
    tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt").to(device)
    summary_ids = ''
    if type==0:
      #Normal
      summary_ids = model.generate(tokenized_text,
                                        num_beams=6,
                                        no_repeat_ngram_size=1,
                                        min_length=min_len,
                                        max_length=max_len,
                                        early_stopping=True)
    elif type==1:
      #Normal
      summary_ids = model.generate(tokenized_text,
                                        num_beams=6,
                                        no_repeat_ngram_size=2,
                                        min_length=min_len,
                                        max_length=max_len,
                                        early_stopping=True)
    elif type==2:
      #Greedy
      summary_ids = model.generate(tokenized_text, min_length=min_len, max_length=max_len)

    elif type==3:
      #Large Beam
      summary_ids = model.generate(tokenized_text,
                                      num_beams=30,
                                      no_repeat_ngram_size=2,
                                      min_length=min_len,
                                      max_length=max_len,
                                      early_stopping=True)
    elif type==4:
      #Sampling
      summary_ids = model.generate(tokenized_text,       
                                      do_sample=True,
                                      min_length=min_len,
                                      max_length=max_len,
                                      top_k=0)
    elif type==5:
      #Sampling with Temp cooled softmax
      summary_ids = model.generate(tokenized_text,       
                                      do_sample=True,
                                      min_length=min_len,
                                      max_length=max_len,
                                      top_k=0,
                                 temperature=0.7)
    elif type==6:
      #Top k
      summary_ids = model.generate(tokenized_text,       
                                      do_sample=True,
                                      min_length=min_len,
                                      max_length=max_len,
                                      top_k=50)
    elif type==7:
      #Top p
      summary_ids = model.generate(tokenized_text, 
                                      do_sample=True,
                                      min_length=min_len,
                                      max_length=max_len,
                                      top_p=0.92,
                                   top_k=0, )

    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    pegasus_summaries.append(output)
  return pegasus_summaries

In [ ]:
for i in [3,5]:
  file = filenames[0]
  summ_data = pd.read_csv(file, header=0, index_col=0)
  index = list(pd.read_csv('index.csv', header=0, index_col=0)['index'])
  pegasus_summ_data = summ_data[['tweet', 'claim_reviewed']].copy()
  pegasus_summ_data.columns = ['source', 'target']
  pegasus_summ_data = pegasus_summ_data.reindex(index)

  kf = KFold(n_splits=5)
  ct = 0
  hf_model_name = 'sshleifer/distill-pegasus-cnn-16-4'
  # %load_ext tensorboard
  # %tensorboard --logdir output/

  for train_index, val_index in kf.split(pegasus_summ_data):
    tokenizer = PegasusTokenizerFast.from_pretrained(hf_model_name)
    model = PegasusForConditionalGeneration.from_pretrained(hf_model_name)

    def tokenize(batch):
      tokenized_input = tokenizer(batch['source'], padding='longest')
      tokenized_label = tokenizer(batch['target'], padding='longest')

      tokenized_input['labels'] = tokenized_label['input_ids']

      return tokenized_input

    train_dataset = Dataset.from_pandas(pegasus_summ_data.iloc[train_index])
    train_dataset = train_dataset.map(tokenize, batched=True, batch_size = len(train_dataset))
    val_dataset = Dataset.from_pandas(pegasus_summ_data.iloc[val_index])
    val_dataset = val_dataset.map(tokenize, batched=True, batch_size = len(val_dataset))
    
    
    train_dataset.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
    val_dataset.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

    model_dir = str(i)+'/model/'+str(ct)+"/"
    output_dir = str(i)+'/output/'+str(ct)+"/"
    now = datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
    logging_dir = output_dir+"runs/"+now+"/"
    

    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        num_train_epochs=16,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
        prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
        learning_rate=0.0002,
        evaluation_strategy='steps', # Run evaluation every eval_steps
        save_steps=100, # How often to save a checkpoint
        save_total_limit=1, # Number of maximum checkpoints to save
        remove_unused_columns=True, # Removes useless columns from the dataset
        run_name='train1', # Wandb run name
        logging_steps=100, # How often to log loss to wandb
        eval_steps=100, # How often to run evaluation on the val_set
        logging_first_step=False, # Whether to log also the very first training step to wandb
        load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
        metric_for_best_model="loss", # Use loss to evaluate best model.
        greater_is_better=False, # Best model is the one with the lowest loss, not highest.
        generation_max_length = 15,
        generation_num_beams = 6,
        predict_with_generate=True,
        report_to="tensorboard",
        logging_dir=logging_dir
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    trainer.train()
    trainer.save_model(model_dir)



    cur_model = PegasusForConditionalGeneration.from_pretrained(model_dir).to(device)

    decoded_sources = val_dataset['source']

    decoded_preds = pegasus_summarizer(val_dataset['source'],cur_model, 5, 15, i)
    decoded_labels = val_dataset['target']

    output = pd.DataFrame({'ID': val_dataset['__index_level_0__'], 'Source Text': decoded_sources, 'Target Text': decoded_labels, 'Generated Text': decoded_preds})
    output.to_csv(output_dir +"/predictions.xlsx")

    del cur_model
    del decoded_preds
    del model
    del tokenizer

    ct = ct +1

  output_dir = str(i)+'/output/'
  prediction_file_paths = ['0/predictions.xlsx', '1/predictions.xlsx', '2/predictions.xlsx', '3/predictions.xlsx', '4/predictions.xlsx']
  for file in prediction_file_paths:
    preds = pd.read_csv(output_dir+file, header=0, index_col=0)
    gold_urls = []
    for index, row in preds.iterrows():
      gold_urls.append(summ_data.loc[row['ID']]['evidence_url'])

    print("{} Pegasus FT Retrieval = {}".format(file, retrieval_eval(preds['Generated Text'], gold_urls)))



  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the training set don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: source, target, __index_level_0__. If source, target, __index_level_0__ are not expected by `PegasusForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 453
  Num Epochs = 16
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 912
/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:131: UserWarning: Creating a tensor from a list of n

Step,Training Loss,Validation Loss
100,6.177000,1.697540
200,1.021400,1.404456
300,0.694600,1.372182
400,0.497500,1.404132
500,0.377400,1.451498
600,0.299300,1.533690
700,0.253900,1.553215
800,0.214800,1.613327
900,0.191100,1.621441


The following columns in the evaluation set don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: source, target, __index_level_0__. If source, target, __index_level_0__ are not expected by `PegasusForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 114
  Batch size = 8
Saving model checkpoint to 3/output/0/checkpoint-100
Configuration saved in 3/output/0/checkpoint-100/config.json
Model weights saved in 3/output/0/checkpoint-100/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: source, target, __index_level_0__. If source, target, __index_level_0__ are not expected by `PegasusForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 114
  Batch size = 8
Saving model checkpoint to 3/output/0/che

...

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The following columns in the training set don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: source, target, __index_level_0__. If source, target, __index_level_0__ are not expected by `PegasusForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 454
  Num Epochs = 16
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 912


Step,Training Loss,Validation Loss
100,6.087700,1.731361
200,1.079700,1.376900
300,0.726900,1.325822
400,0.540000,1.290548
500,0.401500,1.340700
600,0.331600,1.379951
700,0.262500,1.463733
800,0.247700,1.471828
900,0.220000,1.484337


The following columns in the evaluation set don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: source, target, __index_level_0__. If source, target, __index_level_0__ are not expected by `PegasusForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 113
  Batch size = 8
Saving model checkpoint to 5/output/4/checkpoint-100
Configuration saved in 5/output/4/checkpoint-100/config.json
Model weights saved in 5/output/4/checkpoint-100/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: source, target, __index_level_0__. If source, target, __index_level_0__ are not expected by `PegasusForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 113
  Batch size = 8
Saving model checkpoint to 5/output/4/che

0/predictions.xlsx Pegasus FT Retrieval = {'retrieval_1': 24.56140350877193, 'retrieval_5': 34.21052631578947, 'retrieval_10': 34.21052631578947, 'retrieval_20': 34.21052631578947, 'retrieval_50': 34.21052631578947, 'mrr': 0.2880116959064327}
1/predictions.xlsx Pegasus FT Retrieval = {'retrieval_1': 21.92982456140351, 'retrieval_5': 29.82456140350877, 'retrieval_10': 29.82456140350877, 'retrieval_20': 29.82456140350877, 'retrieval_50': 29.82456140350877, 'mrr': 0.2546783625730994}
2/predictions.xlsx Pegasus FT Retrieval = {'retrieval_1': 16.8141592920354, 'retrieval_5': 21.238938053097346, 'retrieval_10': 22.123893805309734, 'retrieval_20': 22.123893805309734, 'retrieval_50': 22.123893805309734, 'mrr': 0.18761061946902655}
3/predictions.xlsx Pegasus FT Retrieval = {'retrieval_1': 17.699115044247787, 'retrieval_5': 26.548672566371682, 'retrieval_10': 26.548672566371682, 'retrieval_20': 26.548672566371682, 'retrieval_50': 26.548672566371682, 'mrr': 0.21489675516224188}
4/predictions.xlsx